## Simulation Experiments of mbst and Projected mbst

In [1]:
import pandas as pd
import numpy as np
import time
import math

## mbst objects
from mbst import *
## training functions
from train import *
## functions generating experiment cases
from exp import *

/Users/hanqiu/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### generating experiment data

In [2]:
N_sample = 100000 ## sample size
N_feats = 10 ## number of feature
N_trees_real = 100 ## number of boosted trees in the random mbst

np.random.seed(32) ## fixed seed for replications

## different specifications of base effect and treatment effects
exp_case = 1
if exp_case == 1:
    ## 1 * base effect + 0.25 * treatment effect
    theta_dim = 2
    theta_factors = np.array([1,0.25])
    base_effect_flag = True
elif exp_case == 2:
    ## 1 * base effect + 1 * treatment effect
    theta_dim = 2
    theta_factors = np.array([1,1])
    base_effect_flag = True
elif exp_case == 3:
    ## 1 * treatment effect (1) + 0.5 * treatment effect (2)
    theta_dim = 2
    theta_factors = np.array([1,0.5])
    base_effect_flag = False
elif exp_case == 4:
    ## 1 * base effect + 1 * treatment effect (1) + 0.5 * treatment effect (2)
    theta_dim = 3
    theta_factors = np.array([1,1,0.5])
    base_effect_flag = True
elif exp_case == 5:
    ## 1 * base effect + 0.5 * treatment effect (1) + 0.25 * treatment effect (2)
    theta_dim = 3
    theta_factors = np.array([1,0.5,0.25])
    base_effect_flag = True

In [3]:
## generate experiment data according to parameters above
feat_list = [str(i) for i in range(N_feats)]
feat_dict = dict([(feat,idx) for idx,feat in enumerate(feat_list)])
X_ext = np.random.normal(size=(N_sample,len(feat_list)))
df_ext = pd.DataFrame(X_ext,columns=feat_list)
X_treat = np.random.normal(size=(N_sample,theta_dim))
if base_effect_flag:
    X_treat[:,0] = 1
_,_,splits_dict = find_splits_dict(df_ext,feat_list)
mbst_real = generate_random_theta_mbst(N_trees_real,feat_list,splits_dict,theta_dim,theta_factors)
theta_real = mbst_real.predict(X_ext,feat_dict)
y_real = np.sum(X_treat * theta_real,axis=-1)

### applying different estimation methods

#### 0. setting common training params

In [4]:
N_trees_train = 300 ## number of boosted trees during training
N_train = int(N_sample * 0.7) ## training samples

params = {'lam_reg':1,
          'learning_rate':0.1,
          'max_depth':4,
          'min_split_loss':0,
          'subsample':0.7,
          'min_childs':10}

params_xgb = {'reg_lambda':params.get('lam_reg',1),
              'learning_rate':params.get('learning_rate',0.1),
              'max_depth':params.get('max_depth',4),
              'subsample':params.get('subsample',0.7),
              'objective':'reg:squarederror'}

def linear_obj(theta,labels):
    ## MSE loss and corresponding grad / hess
    X_treat,y = labels
    y_pred = np.sum(X_treat*theta,axis=-1)
    ll = np.square(y_pred - y)
    loss = np.sqrt(np.mean(ll))
    grad = (y_pred - y).reshape((-1,1)) * X_treat
    hess = np.expand_dims(X_treat,axis=1) * np.expand_dims(X_treat,axis=2)
    return loss,grad,hess

#### 1.vanilla xgb

In [5]:
X_all = np.concatenate([X_ext,X_treat],axis=-1)
xg_dtrain = xgb.DMatrix(X_all[:N_train],label=y_real[:N_train])
xg_dall = xgb.DMatrix(X_all,label=y_real)

tt = time.time()
xgb_bst = xgb.train(params_xgb,xg_dtrain,num_boost_round=N_trees_train,verbose_eval=False)
print('train time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
y_pred = xgb_bst.predict(xg_dall)
print('y predict time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
X_all_new = np.concatenate([X_ext,X_treat],axis=-1)

N_treat_sample = 100 ## for local linear regression
X_treat_samples = X_treat[np.random.choice(len(X_treat),N_treat_sample,replace=False)]
y_pred_new = []
for X_treat_sample in X_treat_samples:
    X_all_new[:,-theta_dim:] = X_treat_sample
    y_pred_new.append(xgb_bst.predict(xgb.DMatrix(X_all_new)))
y_pred_new = np.stack(y_pred_new,axis=0)
A = np.matmul(X_treat_samples.T,X_treat_samples)
b = np.matmul(X_treat_samples.T,y_pred_new)
theta_pred = np.linalg.solve(A,b).T
print('theta predict time: {:.2f}s.'.format(time.time() - tt))

theta_pred_xgb_base,y_pred_xgb_base = theta_pred,y_pred

train time: 5.74s.
y predict time: 0.14s.
theta predict time: 15.99s.


#### 2. multi-stage xgb

In [6]:
X_ext_train = X_ext[:N_train]
X_treat_train = X_treat[:N_train]
y_remain_train = y_real[:N_train].copy()
xg_dall = xgb.DMatrix(X_ext)

theta_pred = []
for iter_ in range(theta_dim):
    header = 'stage {} '.format(iter_)
    w = X_treat_train[:,iter_]
    valid_idxs = (np.abs(w) > 1e-3)
    xg_dtrain = xgb.DMatrix(X_ext_train[valid_idxs],label=y_remain_train[valid_idxs]/w[valid_idxs],
                            weight=w[valid_idxs]**2)
    
    tt = time.time()
    xgb_bst = xgb.train(params_xgb,xg_dtrain,
                        num_boost_round=int(N_trees_train // theta_dim) + 1,verbose_eval=False)
    print(header + 'train time: {:.2f}s.'.format(time.time() - tt))

    tt = time.time()
    theta_iter_pred = xgb_bst.predict(xg_dall)
    theta_pred.append(theta_iter_pred)
    y_remain_train -= w * theta_iter_pred[:N_train]
    print(header + 'predict time: {:.2f}s.'.format(time.time() - tt))
    
theta_pred = np.stack(theta_pred,axis=-1)
y_pred = np.sum(X_treat*theta_pred,axis=-1)

theta_pred_xgb_mstage,y_pred_xgb_mstage = theta_pred,y_pred

stage 0 train time: 3.06s.
stage 0 predict time: 0.07s.
stage 1 train time: 3.06s.
stage 1 predict time: 0.07s.


#### 3. grf

In [7]:
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
from rpy2.robjects import r as rapi
numpy2ri.activate()
grf = importr('grf')

In [8]:
X_ext_train = X_ext[:N_train]
X_treat_train = X_treat[:N_train]
y_remain_train = np.expand_dims(y_real[:N_train].copy(),axis=-1)

theta_pred = []
for iter_ in range(theta_dim):
    header = 'stage {} '.format(iter_)
    w = X_treat_train[:,[iter_]]
    valid_idxs = (np.abs(w[:,0]) > 1e-3)
    
    tt = time.time()
    if np.std(w) < 1e-3:
        grf_bst = grf.regression_forest(X_ext_train[valid_idxs],y_remain_train[valid_idxs],
                               num_trees=int(N_trees_train // theta_dim) + 1)
    else:
        grf_bst = grf.causal_forest(X_ext_train[valid_idxs],y_remain_train[valid_idxs],
                               w[valid_idxs],num_trees=int(N_trees_train // theta_dim) + 1)
    print(header + 'train time: {:.2f}s.'.format(time.time() - tt))

    tt = time.time()
    theta_iter_pred = rapi.predict(grf_bst,X_ext)['predictions']
    theta_pred.append(theta_iter_pred)
    y_remain_train -= w * np.expand_dims(theta_iter_pred[:N_train],axis=-1)
    print(header + 'predict time: {:.2f}s.'.format(time.time() - tt))
    
theta_pred = np.stack(theta_pred,axis=-1)
y_pred = np.sum(X_treat*theta_pred,axis=-1)

theta_pred_grf,y_pred_grf = theta_pred,y_pred

stage 0 train time: 17.75s.
stage 0 predict time: 3.61s.
stage 1 train time: 46.80s.
stage 1 predict time: 2.95s.


#### 4. mbst

In [9]:
tt = time.time()
mbst,_ = train_multi_bst(params,df_ext.iloc[:N_train].copy(),labels=(X_treat[:N_train],y_real[:N_train]),
                         feat_list=feat_list,num_boost_round=N_trees_train,
                         obj_func=linear_obj,verbose=False)
print('train time: {:.2f}s.'.format(time.time() - tt))

tt = time.time()
theta_pred = mbst.predict(X_ext,feat_dict)
y_pred = np.sum(X_treat*theta_pred,axis=-1)
print('predict time: {:.2f}s.'.format(time.time() - tt))

theta_pred_mbst,y_pred_mbst = theta_pred,y_pred

train time: 131.89s.
predict time: 2.43s.


#### 5. projected mbst

In [10]:
## first, define a variety of direction finding methods
def find_direc_func_null(grad,hess,grad_func,mbst,train_info):
    dim = grad.shape[-1]
    direc = np.ones((dim,)) / dim
    return direc

def find_direc_func_grad(grad,hess,grad_func,mbst,train_info):
    direc = find_max_var_direc(grad)
    return direc

def find_direc_func_approx(grad,hess,grad_func,mbst,train_info):
    H_sum = np.sum(hess,axis=0)
    dtheta_est = np.linalg.solve(H_sum,grad.T).T
    direc = find_max_var_direc(dtheta_est)
    return direc

def find_direc_func_nag(grad,hess,grad_func,mbst,train_info,moment_beta=0.9):
    moment = moment_beta * train_info.get('grad_prev',np.zeros(grad.shape))
    grad_new,hess_new = grad_func(train_info.get('curr_theta',np.zeros(grad.shape)) \
                                  + train_info.get('delta_theta_mean',np.zeros(grad.shape)) \
                                  + moment * mbst.learning_rate)
    grad_new = grad_new + moment
    direc = find_max_var_direc(grad_new)
    return direc

In [11]:
mbst_projs = []
theta_pred_projs = []
y_pred_projs = []
name_projs = []

find_direc_funcs = [
#                  ('n',find_direc_func_null),
                 ('g',find_direc_func_grad),
                ('hg',find_direc_func_approx),
                 ('nag',find_direc_func_nag),
                ]

## then, iterate all possible projected mbst options
for corrected_name,corrected in [('n',False),('pc',True)]:
#     for centered in [('nc',False),('c',True)]:
    for centered_name,centered in [('c',True)]:
        for direc_name,find_direc_func in (find_direc_funcs):
            name = 'project mbst ({},{},{})'.format(corrected_name,centered_name,direc_name)
            print(name)
            
            tt = time.time()
            mbst,_ = train_project_bst(params,df_ext.iloc[:N_train].copy(),
                                     labels=(X_treat[:N_train],y_real[:N_train]),
                                     feat_list=feat_list,num_boost_round=N_trees_train,
                                       find_direc_func=find_direc_func,
                                       corrected=corrected,centered=centered,
                                     obj_func=linear_obj,verbose=False)
            print('train time: {:.2f}s.'.format(time.time() - tt))

            tt = time.time()
            theta_pred = mbst.predict(X_ext,feat_dict)
            y_pred = np.sum(X_treat*theta_pred,axis=-1)
            print('predict time: {:.2f}s.'.format(time.time() - tt))
            
            name_projs.append(name)
            mbst_projs.append(mbst)
            theta_pred_projs.append(theta_pred)
            y_pred_projs.append(y_pred)

project mbst (n,c,g)
train time: 46.94s.
predict time: 2.68s.
project mbst (n,c,hg)
train time: 50.17s.
predict time: 2.46s.
project mbst (n,c,nag)
train time: 52.90s.
predict time: 3.03s.
project mbst (pc,c,g)
train time: 107.54s.
predict time: 2.46s.
project mbst (pc,c,hg)
train time: 101.84s.
predict time: 2.50s.
project mbst (pc,c,nag)
train time: 104.65s.
predict time: 2.45s.


### evaluations

In [12]:
def rounding(l,decimals=4):
    return list(np.round(l,decimals=decimals))

method_names = ['mean baseline','xgb','multi-stage xgb','grf','mbst'] + name_projs
y_preds = [np.mean(y_real),y_pred_xgb_base,y_pred_xgb_mstage,y_pred_grf,y_pred_mbst] + y_pred_projs
theta_preds = [np.mean(theta_real,axis=0),theta_pred_xgb_base,theta_pred_xgb_mstage,
               theta_pred_grf,theta_pred_mbst] \
                + theta_pred_projs

y_train_rmse = [np.sqrt(np.mean(np.square(y_pred - y_real)[:N_train])) for y_pred in y_preds]
y_test_rmse = [np.sqrt(np.mean(np.square(y_pred - y_real)[N_train:])) for y_pred in y_preds]
theta_train_rmse = [np.sqrt(np.mean(np.square(theta_pred - theta_real)[:N_train],axis=0)) 
                    for theta_pred in theta_preds]
theta_test_rmse = [np.sqrt(np.mean(np.square(theta_pred - theta_real)[N_train:],axis=0)) 
                   for theta_pred in theta_preds]
theta_train_corr = [np.zeros((theta_dim,))] + \
                    [np.array([np.corrcoef(theta_pred[:,col][:N_train], 
                                          theta_real[:,col][:N_train])[0,1] for col in range(theta_dim)]) 
                    for theta_pred in theta_preds[1:]]
theta_test_corr = [np.zeros((theta_dim,))] + \
                    [np.array([np.corrcoef(theta_pred[:,col][N_train:], 
                                          theta_real[:,col][N_train:])[0,1] for col in range(theta_dim)]) 
                    for theta_pred in theta_preds[1:]]

err_df = pd.DataFrame({'y rmse train':rounding(y_train_rmse),'y rmse test':rounding(y_test_rmse),
                   'theta rmse train':rounding(theta_train_rmse),'theta rmse test':rounding(theta_test_rmse),
                   'theta corr train':rounding(theta_train_corr),'theta corr test':rounding(theta_test_corr),})
err_df = err_df.T
err_df.columns = method_names
display(err_df)

,mean baseline,xgb,multi-stage xgb,grf,mbst,"project mbst (n,c,g)","project mbst (n,c,hg)","project mbst (n,c,nag)","project mbst (pc,c,g)","project mbst (pc,c,hg)","project mbst (pc,c,nag)"
y rmse train,0.0869,0.0312,0.0333,0.0416,0.028,0.034,0.0385,0.0344,0.0315,0.0316,0.0313
y rmse test,0.0865,0.0328,0.035,0.0476,0.0293,0.0355,0.0401,0.036,0.0329,0.0331,0.0329
theta rmse train,"[0.0836, 0.0224]","[0.028, 0.0152]","[0.033, 0.009]","[0.041, 0.0139]","[0.0268, 0.0099]","[0.0339, 0.0089]","[0.0387, 0.0092]","[0.0344, 0.0088]","[0.031, 0.0089]","[0.0312, 0.0089]","[0.0309, 0.0089]"
theta rmse test,"[0.0831, 0.0224]","[0.0288, 0.0153]","[0.0338, 0.0092]","[0.0454, 0.0141]","[0.0275, 0.01]","[0.0343, 0.0091]","[0.039, 0.0093]","[0.0348, 0.009]","[0.0316, 0.0091]","[0.0318, 0.009]","[0.0316, 0.0091]"
theta corr train,"[0.0, 0.0]","[0.9439, 0.7777]","[0.9227, 0.9189]","[0.8816, 0.8187]","[0.9489, 0.8985]","[0.9174, 0.9182]","[0.8904, 0.9132]","[0.9149, 0.9198]","[0.9312, 0.9193]","[0.9305, 0.92]","[0.9319, 0.9202]"
theta corr test,"[0.0, 0.0]","[0.9394, 0.7778]","[0.9177, 0.9155]","[0.8441, 0.8086]","[0.945, 0.8965]","[0.914, 0.9154]","[0.887, 0.9109]","[0.9111, 0.9167]","[0.9274, 0.9162]","[0.9265, 0.9172]","[0.9277, 0.9161]"
